# YogaPartner

Our datasets are in
- `.\data\v1`
- `.\data\v2`

Our frontend code is in `.\android`

Our generated data goes in `.\intermediates`

In [1]:
import os
import matplotlib.pyplot as plt
from pathlib import Path

version = 2

## Global functions

In [2]:
def getPath():
    return r".\data\v" + str(version)

def getClasses():
    path = getPath()
    return os.listdir(path)

def getPathOfClass(className):
    path = getPath()
    return os.path.join(path, className)

## Dataset stats

In [3]:
def visualizeDataSet():
    classFolders = getClasses()
    counts = []
    imageTypes = {}
    for classFolder in classFolders:
        images = os.listdir(getPathOfClass(classFolder))
        count = len(images)
        counts.append(count)
        for image in images:
            image_path = os.path.join(getPathOfClass(classFolder), image)
            p = Path(image_path)
            ext = p.suffix
            imageTypes[ext] = imageTypes.get(ext, 0) + 1

    print("Dataset frequency distribution")
    for i in range(len(classFolders)):
        className = classFolders[i]
        count = counts[i]
        print(className + " : " + str(count))

    print()

    print("Image type frequency distribution")
    for key in imageTypes.keys():
        count = imageTypes[key]
        print(key + " : " + str(count))

    print()
    
visualizeDataSet()

Dataset frequency distribution
adho_mukha_shvanasana : 435
bhujangasana : 668
bidalasana : 315
phalakasana : 145
ustrasana : 66
utkatasana : 226
utkata_konasana : 209
vasisthasana : 148
virabhadrasana_i : 221
virabhadrasana_ii : 389
vrikshasana : 315

Image type frequency distribution
.jpg : 3094
.jpeg : 43



## Cleaning Data

### Convert to Jpeg

In [4]:
from PIL import Image
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    for f in files:
        old_path = os.path.join(class_path, f)
        p = Path(old_path)
        ext = p.suffix
        if ext == ".webp":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jepg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path, "jepg")
            print("Removing " + f)
            os.remove(old_path)
        if ext == ".png":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jpg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path)
            print("Removing " + f)
            os.remove(old_path)
        if ext == ".JPG":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jpg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path)
            print("Removing " + f)
            os.remove(old_path)
print("Converted Webp and PNG")

Converted Webp and PNG


### Finding duplicate images

In [ ]:
from difPy import dif
import tqdm

classFolders = getClasses()
for className in tqdm.tqdm(classFolders, position=0):
    class_path = getPathOfClass(className)
    search = dif(class_path, delete=True)
print("Completed duplicate image search")

### Cleaning up folder names

In [15]:
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    old_path = getPathOfClass(className)
    new_name = re.sub(r"\s+", "_", className)
    if new_name != className:
        print("Renaming " + className)
    new_path = os.path.join(getPath(), new_name)
    os.rename(old_path, new_path)
print("Renamed class names")

Renamed class names


### Removing non supported images

In [16]:
from PIL import Image
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    count = 1
    for f in files:
        f_path = os.path.join(class_path, f)
        if not (f.endswith(".jpg") or f.endswith(".jpeg")):
            os.remove(f_path)
            print("Removed " + f)
print("Finished removing non suporrted images")

Finished removing non suporrted images


### Cleaning up file names

In [22]:
from PIL import Image
import os
import shutil
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    count = 1
    for f in files:
        old_path = os.path.join(class_path, f)
        p = Path(old_path)
        new_file_name = f"{className}_{count}{p.suffix}"
        if new_file_name != f:
            print("Renaming " + f)
        new_file_path = os.path.join(class_path, new_file_name)
        if(os.path.exists(new_file_path)):
            print("Clash in names, change to unique pattern")
            break
        os.rename(old_path, new_file_path)
        count += 1
        
print("Finished cleaning file names")

Renaming adho_mukha_shvanasana_sharkaboi1.jpg
Renaming adho_mukha_shvanasana_sharkaboi10.jpg
Renaming adho_mukha_shvanasana_sharkaboi100.jpg
Renaming adho_mukha_shvanasana_sharkaboi101.jpg
Renaming adho_mukha_shvanasana_sharkaboi102.jpg
Renaming adho_mukha_shvanasana_sharkaboi103.jpg
Renaming adho_mukha_shvanasana_sharkaboi104.jpg
Renaming adho_mukha_shvanasana_sharkaboi105.jpg
Renaming adho_mukha_shvanasana_sharkaboi106.jpg
Renaming adho_mukha_shvanasana_sharkaboi107.jpg
Renaming adho_mukha_shvanasana_sharkaboi108.jpg
Renaming adho_mukha_shvanasana_sharkaboi109.jpg
Renaming adho_mukha_shvanasana_sharkaboi11.jpg
Renaming adho_mukha_shvanasana_sharkaboi110.jpg
Renaming adho_mukha_shvanasana_sharkaboi111.jpg
Renaming adho_mukha_shvanasana_sharkaboi112.jpg
Renaming adho_mukha_shvanasana_sharkaboi113.jpg
Renaming adho_mukha_shvanasana_sharkaboi114.jpg
Renaming adho_mukha_shvanasana_sharkaboi115.jpg
Renaming adho_mukha_shvanasana_sharkaboi116.jpg
Renaming adho_mukha_shvanasana_sharkaboi117.

Renaming adho_mukha_shvanasana_sharkaboi384.jpg
Renaming adho_mukha_shvanasana_sharkaboi385.jpg
Renaming adho_mukha_shvanasana_sharkaboi386.jpg
Renaming adho_mukha_shvanasana_sharkaboi387.jpg
Renaming adho_mukha_shvanasana_sharkaboi388.jpg
Renaming adho_mukha_shvanasana_sharkaboi389.jpg
Renaming adho_mukha_shvanasana_sharkaboi39.jpg
Renaming adho_mukha_shvanasana_sharkaboi390.jpg
Renaming adho_mukha_shvanasana_sharkaboi391.jpg
Renaming adho_mukha_shvanasana_sharkaboi392.jpg
Renaming adho_mukha_shvanasana_sharkaboi393.jpg
Renaming adho_mukha_shvanasana_sharkaboi394.jpg
Renaming adho_mukha_shvanasana_sharkaboi395.jpg
Renaming adho_mukha_shvanasana_sharkaboi396.jpg
Renaming adho_mukha_shvanasana_sharkaboi397.jpg
Renaming adho_mukha_shvanasana_sharkaboi398.jpg
Renaming adho_mukha_shvanasana_sharkaboi399.jpg
Renaming adho_mukha_shvanasana_sharkaboi4.jpg
Renaming adho_mukha_shvanasana_sharkaboi40.jpg
Renaming adho_mukha_shvanasana_sharkaboi400.jpg
Renaming adho_mukha_shvanasana_sharkaboi401.

Renaming bhujangasana_sharkaboi27.jpg
Renaming bhujangasana_sharkaboi270.jpg
Renaming bhujangasana_sharkaboi271.jpg
Renaming bhujangasana_sharkaboi272.jpg
Renaming bhujangasana_sharkaboi273.jpg
Renaming bhujangasana_sharkaboi274.jpg
Renaming bhujangasana_sharkaboi275.jpg
Renaming bhujangasana_sharkaboi276.jpg
Renaming bhujangasana_sharkaboi277.jpg
Renaming bhujangasana_sharkaboi278.jpg
Renaming bhujangasana_sharkaboi279.jpg
Renaming bhujangasana_sharkaboi28.jpg
Renaming bhujangasana_sharkaboi280.jpg
Renaming bhujangasana_sharkaboi281.jpg
Renaming bhujangasana_sharkaboi282.jpg
Renaming bhujangasana_sharkaboi283.jpg
Renaming bhujangasana_sharkaboi284.jpg
Renaming bhujangasana_sharkaboi285.jpg
Renaming bhujangasana_sharkaboi286.jpg
Renaming bhujangasana_sharkaboi287.jpg
Renaming bhujangasana_sharkaboi288.jpg
Renaming bhujangasana_sharkaboi289.jpg
Renaming bhujangasana_sharkaboi29.jpg
Renaming bhujangasana_sharkaboi290.jpg
Renaming bhujangasana_sharkaboi291.jpg
Renaming bhujangasana_sharka

Renaming bhujangasana_sharkaboi579.jpg
Renaming bhujangasana_sharkaboi58.jpg
Renaming bhujangasana_sharkaboi580.jpg
Renaming bhujangasana_sharkaboi581.jpg
Renaming bhujangasana_sharkaboi582.jpg
Renaming bhujangasana_sharkaboi583.jpg
Renaming bhujangasana_sharkaboi584.jpg
Renaming bhujangasana_sharkaboi585.jpg
Renaming bhujangasana_sharkaboi586.jpg
Renaming bhujangasana_sharkaboi587.jpg
Renaming bhujangasana_sharkaboi588.jpg
Renaming bhujangasana_sharkaboi589.jpg
Renaming bhujangasana_sharkaboi59.jpg
Renaming bhujangasana_sharkaboi590.jpg
Renaming bhujangasana_sharkaboi591.jpg
Renaming bhujangasana_sharkaboi592.jpg
Renaming bhujangasana_sharkaboi593.jpg
Renaming bhujangasana_sharkaboi594.jpg
Renaming bhujangasana_sharkaboi595.jpg
Renaming bhujangasana_sharkaboi596.jpg
Renaming bhujangasana_sharkaboi597.jpg
Renaming bhujangasana_sharkaboi598.jpg
Renaming bhujangasana_sharkaboi599.jpg
Renaming bhujangasana_sharkaboi6.jpg
Renaming bhujangasana_sharkaboi60.jpg
Renaming bhujangasana_sharkabo

Renaming bidalasana_sharkaboi175.jpg
Renaming bidalasana_sharkaboi176.jpg
Renaming bidalasana_sharkaboi177.jpg
Renaming bidalasana_sharkaboi178.jpg
Renaming bidalasana_sharkaboi179.jpg
Renaming bidalasana_sharkaboi18.jpg
Renaming bidalasana_sharkaboi180.jpeg
Renaming bidalasana_sharkaboi181.jpg
Renaming bidalasana_sharkaboi182.jpg
Renaming bidalasana_sharkaboi183.jpg
Renaming bidalasana_sharkaboi184.jpg
Renaming bidalasana_sharkaboi185.jpg
Renaming bidalasana_sharkaboi186.jpg
Renaming bidalasana_sharkaboi187.jpg
Renaming bidalasana_sharkaboi188.jpg
Renaming bidalasana_sharkaboi189.jpeg
Renaming bidalasana_sharkaboi19.jpg
Renaming bidalasana_sharkaboi190.jpg
Renaming bidalasana_sharkaboi191.jpg
Renaming bidalasana_sharkaboi192.jpg
Renaming bidalasana_sharkaboi193.jpg
Renaming bidalasana_sharkaboi194.jpg
Renaming bidalasana_sharkaboi195.jpg
Renaming bidalasana_sharkaboi196.jpg
Renaming bidalasana_sharkaboi197.jpg
Renaming bidalasana_sharkaboi198.jpg
Renaming bidalasana_sharkaboi199.jpg
R

...

Renaming utkata_konasana_sharkaboi151.jpeg
Renaming utkata_konasana_sharkaboi152.jpg
Renaming utkata_konasana_sharkaboi153.jpg
Renaming utkata_konasana_sharkaboi154.jpg
Renaming utkata_konasana_sharkaboi155.jpg
Renaming utkata_konasana_sharkaboi156.jpg
Renaming utkata_konasana_sharkaboi157.jpg
Renaming utkata_konasana_sharkaboi158.jpg
Renaming utkata_konasana_sharkaboi159.jpg
Renaming utkata_konasana_sharkaboi16.jpg
Renaming utkata_konasana_sharkaboi160.jpeg
Renaming utkata_konasana_sharkaboi161.jpg
Renaming utkata_konasana_sharkaboi162.jpg
Renaming utkata_konasana_sharkaboi163.jpg
Renaming utkata_konasana_sharkaboi164.jpg
Renaming utkata_konasana_sharkaboi165.jpg
Renaming utkata_konasana_sharkaboi166.jpg
Renaming utkata_konasana_sharkaboi167.jpg
Renaming utkata_konasana_sharkaboi168.jpg
Renaming utkata_konasana_sharkaboi169.jpg
Renaming utkata_konasana_sharkaboi17.jpg
Renaming utkata_konasana_sharkaboi170.jpg
Renaming utkata_konasana_sharkaboi171.jpg
Renaming utkata_konasana_sharkaboi

Renaming vasisthasana_sharkaboi65.jpg
Renaming vasisthasana_sharkaboi66.jpg
Renaming vasisthasana_sharkaboi67.jpg
Renaming vasisthasana_sharkaboi68.jpg
Renaming vasisthasana_sharkaboi69.jpg
Renaming vasisthasana_sharkaboi7.jpg
Renaming vasisthasana_sharkaboi70.jpg
Renaming vasisthasana_sharkaboi71.jpg
Renaming vasisthasana_sharkaboi72.jpg
Renaming vasisthasana_sharkaboi73.jpg
Renaming vasisthasana_sharkaboi74.jpg
Renaming vasisthasana_sharkaboi75.jpg
Renaming vasisthasana_sharkaboi76.jpg
Renaming vasisthasana_sharkaboi77.jpg
Renaming vasisthasana_sharkaboi78.jpg
Renaming vasisthasana_sharkaboi79.jpg
Renaming vasisthasana_sharkaboi8.jpg
Renaming vasisthasana_sharkaboi80.jpg
Renaming vasisthasana_sharkaboi81.jpg
Renaming vasisthasana_sharkaboi82.jpg
Renaming vasisthasana_sharkaboi83.jpg
Renaming vasisthasana_sharkaboi84.jpg
Renaming vasisthasana_sharkaboi85.jpg
Renaming vasisthasana_sharkaboi86.jpg
Renaming vasisthasana_sharkaboi87.jpg
Renaming vasisthasana_sharkaboi88.jpg
Renaming vasis

Renaming virabhadrasana_ii_sharkaboi153.jpg
Renaming virabhadrasana_ii_sharkaboi154.jpg
Renaming virabhadrasana_ii_sharkaboi155.jpg
Renaming virabhadrasana_ii_sharkaboi156.jpg
Renaming virabhadrasana_ii_sharkaboi157.jpg
Renaming virabhadrasana_ii_sharkaboi158.jpg
Renaming virabhadrasana_ii_sharkaboi159.jpg
Renaming virabhadrasana_ii_sharkaboi16.jpg
Renaming virabhadrasana_ii_sharkaboi160.jpg
Renaming virabhadrasana_ii_sharkaboi161.jpg
Renaming virabhadrasana_ii_sharkaboi162.jpg
Renaming virabhadrasana_ii_sharkaboi163.jpg
Renaming virabhadrasana_ii_sharkaboi164.jpg
Renaming virabhadrasana_ii_sharkaboi165.jpg
Renaming virabhadrasana_ii_sharkaboi166.jpg
Renaming virabhadrasana_ii_sharkaboi167.jpg
Renaming virabhadrasana_ii_sharkaboi168.jpg
Renaming virabhadrasana_ii_sharkaboi169.jpg
Renaming virabhadrasana_ii_sharkaboi17.jpg
Renaming virabhadrasana_ii_sharkaboi170.jpg
Renaming virabhadrasana_ii_sharkaboi171.jpg
Renaming virabhadrasana_ii_sharkaboi172.jpg
Renaming virabhadrasana_ii_sharkab

Renaming virabhadrasana_ii_sharkaboi369.jpg
Renaming virabhadrasana_ii_sharkaboi37.jpg
Renaming virabhadrasana_ii_sharkaboi370.jpg
Renaming virabhadrasana_ii_sharkaboi371.jpg
Renaming virabhadrasana_ii_sharkaboi372.jpg
Renaming virabhadrasana_ii_sharkaboi373.jpg
Renaming virabhadrasana_ii_sharkaboi374.jpg
Renaming virabhadrasana_ii_sharkaboi375.jpg
Renaming virabhadrasana_ii_sharkaboi376.jpg
Renaming virabhadrasana_ii_sharkaboi377.jpg
Renaming virabhadrasana_ii_sharkaboi378.jpg
Renaming virabhadrasana_ii_sharkaboi379.jpg
Renaming virabhadrasana_ii_sharkaboi38.jpg
Renaming virabhadrasana_ii_sharkaboi380.jpg
Renaming virabhadrasana_ii_sharkaboi381.jpg
Renaming virabhadrasana_ii_sharkaboi382.jpg
Renaming virabhadrasana_ii_sharkaboi383.jpg
Renaming virabhadrasana_ii_sharkaboi384.jpg
Renaming virabhadrasana_ii_sharkaboi385.jpg
Renaming virabhadrasana_ii_sharkaboi386.jpg
Renaming virabhadrasana_ii_sharkaboi387.jpg
Renaming virabhadrasana_ii_sharkaboi388.jpg
Renaming virabhadrasana_ii_sharkab

Renaming vrikshasana_sharkaboi280.jpg
Renaming vrikshasana_sharkaboi281.jpg
Renaming vrikshasana_sharkaboi282.jpg
Renaming vrikshasana_sharkaboi283.jpg
Renaming vrikshasana_sharkaboi284.jpg
Renaming vrikshasana_sharkaboi285.jpg
Renaming vrikshasana_sharkaboi286.jpg
Renaming vrikshasana_sharkaboi287.jpg
Renaming vrikshasana_sharkaboi288.jpg
Renaming vrikshasana_sharkaboi289.jpg
Renaming vrikshasana_sharkaboi29.jpg
Renaming vrikshasana_sharkaboi290.jpg
Renaming vrikshasana_sharkaboi291.jpg
Renaming vrikshasana_sharkaboi292.jpg
Renaming vrikshasana_sharkaboi293.jpg
Renaming vrikshasana_sharkaboi294.jpg
Renaming vrikshasana_sharkaboi295.jpg
Renaming vrikshasana_sharkaboi296.jpg
Renaming vrikshasana_sharkaboi297.jpg
Renaming vrikshasana_sharkaboi298.jpg
Renaming vrikshasana_sharkaboi299.jpg
Renaming vrikshasana_sharkaboi3.jpg
Renaming vrikshasana_sharkaboi30.jpg
Renaming vrikshasana_sharkaboi300.jpg
Renaming vrikshasana_sharkaboi301.jpg
Renaming vrikshasana_sharkaboi302.jpg
Renaming vriksha

### Removing truncated images

In [6]:
from PIL import Image as pil_image
import time

def remove_file(path, retries=3, sleep=0.1):
    for i in range(retries):
        try:
            os.remove(path)
        except WindowsError:
            time.sleep(sleep)
        else:
            break

path = r".\data\v2"
fols = os.listdir(path)
for f in fols:
    fil = os.listdir(os.path.join(path, f))
    for file in fil:
        fpath = os.path.join(path, f, file)
        img = pil_image.open(fpath)
        try:
            img.getpixel((0,0))
            plt.imread(fpath)
        except OSError as e:
            img.close()
            print("Removing "+ fpath)
            remove_file(fpath)
            
print("Done")

Done


### Cleaning to (h, w, 3) format

In [12]:
path = r".\data\v2"
fols = os.listdir(path)
for f in fols:
    fil = os.listdir(os.path.join(path, f))
    for file in fil:
        fpath = os.path.join(path, f, file)
        img = plt.imread(fpath)
        if not (len(img.shape) == 3 and img.shape[2] == 3):
            print(fpath)
            
print("Done")

Done


In [13]:
visualizeDataSet()

Dataset frequency distribution
adho_mukha_shvanasana : 435
bhujangasana : 668
bidalasana : 315
phalakasana : 145
ustrasana : 66
utkatasana : 226
utkata_konasana : 209
vasisthasana : 148
virabhadrasana_i : 221
virabhadrasana_ii : 389
vrikshasana : 315

Image type frequency distribution
.jpg : 3094
.jpeg : 43



## Pre-process

In [14]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

classFolders = getClasses()
filepaths = []
labels = []

for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    for f in files:
        fpath = os.path.join(class_path, f)
        filepaths.append(fpath)
        labels.append(className)

Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')
df=pd.concat([Fseries, Lseries], axis=1)
trsplit = 0.8
vsplit = 0.1
# split df into train_df and test_df 
dsplit=vsplit/(1-trsplit)
strat=df['labels']    
train_df, dummy_df = train_test_split(df, train_size=trsplit, shuffle=True, random_state=123, stratify=strat)
strat=dummy_df['labels']
valid_df, test_df = train_test_split(dummy_df, train_size=dsplit, shuffle=True, random_state=123, stratify=strat)
print('train_df length: ', len(train_df), '  test_df length: ',len(test_df), '  valid_df length: ', len(valid_df), ' total length: ', len(filepaths))
print(train_df['labels'].value_counts())

train_df length:  2509   test_df length:  314   valid_df length:  314  total length:  3137
bhujangasana             534
adho_mukha_shvanasana    348
virabhadrasana_ii        311
vrikshasana              252
bidalasana               252
utkatasana               181
virabhadrasana_i         177
utkata_konasana          167
vasisthasana             118
phalakasana              116
ustrasana                 53
Name: labels, dtype: int64


## Augmentation

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil

def trim (df, max_size, min_size, column):
    df=df.copy()
    sample_list=[] 
    groups=df.groupby(column)
    for label in df[column].unique():        
        group=groups.get_group(label)
        sample_count=len(group)         
        if sample_count> max_size :
            samples=group.sample(max_size, replace=False, weights=None, random_state=123, axis=0).reset_index(drop=True)
            sample_list.append(samples)
        elif sample_count>= min_size:
            sample_list.append(group)
    df=pd.concat(sample_list, axis=0).reset_index(drop=True)
    balance=list(df[column].value_counts())
    print (balance)
    return df

def balance(train_df,max_samples, min_samples, column, working_dir, image_size):
    train_df=train_df.copy()
    train_df=trim (train_df, max_samples, min_samples, column)    
    # make directories to store augmented images
    aug_dir=os.path.join(working_dir, 'aug')
    if os.path.isdir(aug_dir):
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)
    for label in train_df['labels'].unique():    
        dir_path=os.path.join(aug_dir,label)    
        os.mkdir(dir_path)
    # create and store the augmented images  
    total=0
    gen=ImageDataGenerator(horizontal_flip=True,  rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
    groups=train_df.groupby('labels') # group by class
    for label in train_df['labels'].unique():  # for every class               
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
        sample_count=len(group)   # determine how many samples there are in this class  
        if sample_count< max_samples: # if the class has less than target number of images
            aug_img_count=0
            delta=max_samples-sample_count  # number of augmented images to create
            target_dir=os.path.join(aug_dir, label)  # define where to write the images    
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=image_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                            save_format='jpg')
            while aug_img_count<delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total +=aug_img_count
    print('Total Augmented images created= ', total)
    # create aug_df and merge with train_df to create composite training set ndf
    if total>0:
        aug_fpaths=[]
        aug_labels=[]
        classlist=os.listdir(aug_dir)
        for klass in classlist:
            classpath=os.path.join(aug_dir, klass)     
            flist=os.listdir(classpath)    
            for f in flist:        
                fpath=os.path.join(classpath,f)         
                aug_fpaths.append(fpath)
                aug_labels.append(klass)
        Fseries=pd.Series(aug_fpaths, name='filepaths')
        Lseries=pd.Series(aug_labels, name='labels')
        aug_df=pd.concat([Fseries, Lseries], axis=1)
        ndf=pd.concat([train_df,aug_df], axis=0).reset_index(drop=True)
    else:
        ndf=train_df
    print (list(ndf['labels'].value_counts()) )
    return ndf 

max_samples= 250
min_samples=0
column='labels'
working_dir = r'./'
img_size=(256, 256)
ndf=balance(train_df, max_samples, min_samples, column, working_dir, img_size)

[250, 250, 250, 250, 250, 181, 177, 167, 118, 116, 53]
Found 118 validated image filenames.
Found 177 validated image filenames.
Found 116 validated image filenames.
Found 181 validated image filenames.
Found 167 validated image filenames.
Found 53 validated image filenames.
Total Augmented images created=  688
[250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250]


### Save detected poses of training set for debugging

In [91]:
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

toDir = r".\intermediates\v" + str(version)
                
def saveImagesWithKeypoints(image_path, results):
    image_path = os.path.abspath(image_path)
    try:
        image = cv2.imread(image_path)
        image = image.copy()
        if results.pose_landmarks is not None:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        p = Path(image_path)
        fileName = p.name
        fp = os.path.join(toDir, fileName)
        fp = r""+os.path.abspath(fp)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if not cv2.imwrite(fp, image):
            raise Exception("image not saved to "+fp)
        print("Saved to "+ fp)
    except (Exception) as e:
        print("Failed to write image for " + image_path)
        raise e

### To save landmarks to file

In [92]:
import csv
import numpy as np

landmark_count = 33

def save_to_csv(results, class_name, image_path, image_shape):
    csvPath = os.path.join(toDir, "out.csv")
    with open(csvPath, 'w', newline='\n') as csv_file:
        csv_out_writer = csv.writer(csv_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        
        pose_landmarks = results.pose_landmarks
        if pose_landmarks is not None and len(pose_landmarks.landmark) == landmark_count:
            pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
            # Map pose landmarks from [0, 1] range to absolute coordinates to get correct aspect ratio.
            frame_height, frame_width = image_shape
            pose_landmarks *= np.array([frame_width, frame_height, frame_width])
            pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
            p = Path(image_path)
            fileName = p.name
            csv_out_writer.writerow([fileName, class_name] + pose_landmarks)
        else:
            print('No pose detected - length  - ' + image_path)

## Running pre-trained detection model

In [93]:
import tensorflow as tf
from matplotlib.collections import LineCollection
import tqdm

durations = []

with mp_pose.Pose(
    static_image_mode=True, 
#     min_detection_confidence=0.5, 
#     min_tracking_confidence=0.3,
    model_complexity=0
) as pose:
    for i, value in tqdm.tqdm(train_df.iterrows(), position=0):
        begin = time.time()
        label = value['labels']
        image_path = value['filepaths']
        image = cv2.imread(image_path)
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        end = time.time()
        save_to_csv(results, label, image_path, input_image.shape[:2])
        saveImagesWithKeypoints(image_path, results)
        if results.pose_landmarks is not None:
            durations.append(end - begin)
    
print("Done detecting")
print("Total time taken - " + str(sum(durations)))
print("Max time taken - " + str(max(durations)))
print("Min time taken - " + str(min(durations)))
print("Average time taken - " + str(sum(durations) / len(durations)))

2it [00:00,  3.86it/s]

Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\vrikshasana_121.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\vasisthasana_63.jpg


5it [00:01,  3.28it/s]

Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\vrikshasana_8.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\virabhadrasana_i_158.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\virabhadrasana_ii_177.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\adho_mukha_shvanasana_223.jpg


9it [00:02,  6.81it/s]

...

1747it [03:38, 12.85it/s]

Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\bhujangasana_476.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\virabhadrasana_i_73.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\virabhadrasana_i_157.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\bhujangasana_249.jpg


1748it [03:39,  7.98it/s]

No pose detected - length  - .\data\v2\bidalasana\bidalasana_126.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\bidalasana_126.jpg


KeyboardInterrupt: 

## Train classifier model

In [ ]:
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')


processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])